## FAKE NEWS DETECTOR - LSTM

### Imports

In [3]:
! pip3 install tensorflow
! pip3 install keras
! pip3 install nltk
! pip3 install gensim
! pip3 install scikit-learn
! pip3 install keras-tuner
! pip3 install keras_tuner

#### Others
import itertools
import pandas as pd
import numpy as np
import tensorflow
import re
import nltk
import warnings
warnings.filterwarnings("ignore")

#### Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

#### NLTK
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

#### Tensorflow
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow import keras

#### Keras
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.6 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Constants

In [4]:
DATA_BASE_PATH = "./"
TRAIN_RATIO = 0.70
TEST_RATIO = 0.30
MAX_FEATURES_VECTORIZER= 500
TRAINING_EPOCHS = 20 # Number of training epochs
BATCH_SIZE = 64

VOCABULARY_SIZE = 5000 # the size of the vocabulary, indicating the maximum number of unique words that will be considered during the text embedding process.
SENTENCE_LENGTH = 20 # the desired length of each sentence or text sequence after preprocessing. It indicates the number of words that will be included in each sequence.
EMBEDDING_VECTOR_FEATURES = 40 # the number of dimensions in which each word will be represented in the embedding space. It determines the size of the word vectors generated, where words are mapped to continuous vector representations for machine learning tasks.

### Load data

In [5]:
# Load Dataset
train_data = pd.read_csv(DATA_BASE_PATH + 'train.csv')
test_data = pd.read_csv(DATA_BASE_PATH + 'test.csv')

### Preprocessing

The provided code snippet involves data preprocessing steps.

First of all, we must remove the rows with missing values (NaN) from the train_data dataset.

After dropping rows, the indices of the remaining rows may become non-contiguous by resetting the index of the DataFrame to ensure continuous and sequential indexing. This results in an updated DataFrame with a reset index, where the previous index values are moved to a new column, and a new sequential index is assigned to each row.

In [6]:
# Remove NaN
train_data = train_data.dropna()
train_data.reset_index(inplace = True)

First step to continue preprocessing is to prepare input features and targe labels by creating two dataframes, one of them leaving only the input features for model training, and the other with the 'label' column is assigned to the variable y_train, representing the target labels corresponding to the input features in x_train.

In [7]:
# Get target column (Y) and input features (X)
x_train = train_data.drop('label',axis =1)
y_train = train_data['label']

The provided code snippet initializes a PorterStemmer (from the Natural Language Toolkit (NLTK) library is initialized. The stemmer will be used to reduce words to their root form.) for text stemming and creates empty lists corpus and words. It iterates through each title in the DataFrame, removing non-alphanumeric characters, converting to lowercase, and splitting into words. The words are then stemmed and stopwords are removed, resulting in preprocessed sentences added to the corpus list and individual stemmed words to the words list. This process prepares the text data for analysis or model training.

In [8]:
# Stemming and preprocessing
ps = PorterStemmer()
corpus = []
words = []
for i in range(0,len(train_data)):
    review = re.sub('[^a-zA-Z0-9]',' ',train_data['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    statements = ' '.join(review)
    corpus.append(statements)
    words.append(review)

In the provided code, a copy of the training data x_train is assigned to the variable messages. The reset_index() method is then applied to the messages DataFrame. This method is often used to reset the index of a DataFrame, which means that the current index (usually numeric) is replaced with a default integer index starting from 0.

By calling reset_index() with the inplace=True parameter, the operation modifies the DataFrame messages in place, without creating a new DataFrame. This can be useful when you want to apply changes directly to the existing DataFrame instead of creating a new copy.

It's important to note that the specific impact of resetting the index depends on the structure and context of your data. In some cases, you might need to reset the index to make further manipulations or analyses easier, especially if the original index is not providing meaningful information. However, it's recommended to understand the implications of changing the index before applying it to your data.

In [9]:
messages = x_train.copy()
messages.reset_index(inplace=True)

One-hot encoding is often used as a preprocessing step when working with natural language processing (NLP) tasks like text classification, including the detection of fake news using techniques like LSTM (Long Short-Term Memory) networks. LSTM networks are a type of recurrent neural network (RNN) that can effectively model sequences and patterns in sequential data like text.

One-hot encoding is used with LSTM for the following reasons:

- Input Representation: LSTM networks require input data to be in a numerical format. One-hot encoding converts words into numerical vectors, where each word is represented by a vector with all zeros except for a single "1" at the index corresponding to the word's position in the vocabulary. This allows text data to be fed into the network as numeric sequences.

- Sparse Data Handling: NLP datasets typically have a large vocabulary, resulting in sparse data when using raw text representation. One-hot encoding reduces this sparsity by representing each word as a fixed-size vector. This can make training more efficient and reduce memory requirements.

- Word Relationships: One-hot encoding treats each word as independent, which may not capture the semantic relationships between words. However, LSTM networks can learn contextual information from sequences of one-hot encoded vectors, allowing them to capture word relationships and dependencies within a text.

- Sequence Modeling: LSTM networks excel at modeling sequences, and one-hot encoded vectors provide a suitable input format for sequential data. LSTM cells can maintain and update internal states that help capture longer-range dependencies in text.

- Embedding Layer: In many cases, one-hot encoded vectors are further transformed using an embedding layer within the LSTM network. This layer learns dense representations (word embeddings) that capture semantic relationships between words. These learned embeddings can enhance the model's ability to understand the meaning and context of words.

In the context of detecting fake news, one-hot encoding followed by LSTM modeling allows the network to learn patterns and relationships within the textual data, enabling the model to identify relevant features and make accurate predictions about the authenticity of news articles.

The one_hot() function takes a word and a vocabulary size as input and returns a unique integer value for that word based on its position in the vocabulary. This technique is called one-hot encoding, where each word is represented by a vector with all values as zeros except for the position corresponding to the word's index in the vocabulary, which is set to 1.

The resulting onehot_repr list contains one-hot encoded representations of words for each text document in the corpus. This representation is often used as an initial step before feeding the data into neural networks or other machine learning models for further processing.

In [10]:
onehot_repr = [one_hot(words, VOCABULARY_SIZE) for words in corpus]
onehot_repr

[[473, 3618, 1727, 4573, 2019, 2770, 3378, 3947, 1178, 3403],
 [8, 242, 3024, 242, 4759, 4993, 2314],
 [1187, 3959, 4580, 2571],
 [4446, 1609, 4673, 2916, 4588, 1159, 4028],
 [134, 4759, 3558, 772, 3983, 141, 4759, 3326, 218, 2563],
 [4393,
  4642,
  2184,
  1437,
  4369,
  1876,
  60,
  1093,
  2628,
  1370,
  2640,
  1389,
  4067,
  4808,
  2314],
 [659, 2187, 60, 4623, 1435, 1836, 2045, 2413, 2542, 579, 4691],
 [3334, 1245, 1129, 4616, 4057, 1509, 1876, 1398, 2542, 579, 4691],
 [1408, 2980, 2046, 1480, 4234, 4692, 2493, 4069, 1876, 3967],
 [3876, 2815, 2985, 4315, 4334, 3845, 4843, 880],
 [1062, 3124, 4024, 190, 175, 2550, 2377, 2600, 4022, 4111, 770],
 [4588, 3138, 2019, 4692, 1876, 4057],
 [3501, 4341, 3894, 113, 4432, 9, 4162, 3762, 2901],
 [4345, 136, 1691, 1668, 1971, 669, 2503, 2542, 579, 4691],
 [2917, 3997, 4435, 4484, 4414, 2542, 579, 4691],
 [1957, 3771, 4947, 1761, 3584, 4410, 1342, 1208, 2366, 4942, 2070, 2870],
 [1654, 1706, 242],
 [438, 543, 2229, 2924, 1876, 4137, 249

In the following lines, after encoding (0, 1) the words in the given corpus, the variable embedded_docs is created using the pad_sequences function from Keras. This step is essential when preparing text data for training neural network models like LSTM for natural language processing tasks.

The onehot_repr list contains one-hot encoded representations of words in news article texts, and each inner list corresponds to a sequence. The padding='post' parameter ensures that padding is added to the end of each sequence, and maxlen specifies the desired sequence length after padding. The resulting embedded_docs array contains the transformed and padded sequences, forming the input data for the LSTM model.

This preparation process is vital to maintain consistent sequence lengths required for neural network training and allows the data to be effectively fed into the LSTM model for further analysis and prediction.

In [11]:
embedded_docs = pad_sequences(onehot_repr, padding='post' ,maxlen=SENTENCE_LENGTH)
print(embedded_docs)

[[ 473 3618 1727 ...    0    0    0]
 [   8  242 3024 ...    0    0    0]
 [1187 3959 4580 ...    0    0    0]
 ...
 [2134 3167 1036 ...    0    0    0]
 [3116 4057  536 ...    0    0    0]
 [1012 3637 4315 ...    0    0    0]]


In the provided code and output, the array embedded_docs[0] represents the first sequence of word indices after one-hot encoding and padding. Each number in the array corresponds to the index of a word in the vocabulary, and this sequence has been padded with zeros to match a specified length (maxlen). This format is suitable for input into neural network models, such as LSTM, where each number indicates the word's presence in the text. The zeros represent the padding introduced to ensure uniform sequence lengths across all input samples. This processed array serves as a structured input for the subsequent stages of the LSTM model, enabling the analysis and prediction of the underlying patterns in the text data.

In [12]:
embedded_docs[0]

array([ 473, 3618, 1727, 4573, 2019, 2770, 3378, 3947, 1178, 3403,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

### Neural Network Design

As an option of design, the goal is to create a sequential neural network model for text classification, where the input text sequences are transformed through an embedding layer, passed through an LSTM layer to capture sequence information, and finally output a binary classification prediction. Dropout layers are used to mitigate overfitting during training.

The model includes an embedding layer that transforms integer-encoded words into fixed-size vectors, followed by dropout layers to prevent overfitting.
- An LSTM layer with 100 units captures sequential information, and another dropout layer is employed for regularization.
- A dense layer with a sigmoid activation produces the final binary classification output.
- The model is compiled with binary cross-entropy loss, the Adam optimizer, and accuracy as the evaluation metric.

The model's architecture summary is printed, revealing layer configurations and parameter counts. This architecture enables the model to process input text data, capture context through LSTM, and make binary classification predictions.

In [13]:
model=Sequential()
model.add(Embedding(VOCABULARY_SIZE,EMBEDDING_VECTOR_FEATURES,input_length=SENTENCE_LENGTH))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


### Tranining

Now, before training process, we must prepare the input for the neural network.

In the provided code, X_final represents the final numpy array of embedded documents obtained after processing the text data through the one-hot encoding and padding steps. This array contains sequences of word indices, each corresponding to a processed text.

Additionally, Y_final contains the labels associated with the training dataset

This processed data can now be used for testing and evaluating the LSTM model's performance on detecting fake news.

In [14]:
X_final = np.array(embedded_docs)
Y_final = np.array(y_train)

In [15]:
# Data split
X_train_embed, X_test_embed, Y_train_embed, Y_test_embed = train_test_split(X_final, Y_final, test_size=TEST_RATIO, random_state=27)

The provided code snippet trains a sequential neural network model using TensorFlow.

The model architecture consists of an embedding layer followed by a dropout layer, a Long Short-Term Memory (LSTM) layer with 100 units, another dropout layer, and a dense layer with a sigmoid activation function.

The model is compiled with the binary cross-entropy loss function and the Adam optimizer. The training data X_train_embed and Y_train_embed are used for training, and the validation data X_test_embed and Y_test_embed are used for validation. T

The model is trained over 20 epochs, with each batch containing 64 samples. This combination of architecture, loss function, optimizer, and training configuration is designed to achieve high accuracy in binary classification tasks, possibly like detecting fake news or other similar tasks.

The choice of the number of epochs and batch size in training a neural network depends on various factors and needs to be fine-tuned based on the specific problem and dataset.

In the given code, the model is trained using the fit() function with 20 epochs and a batch size of 64.

 - 20 Epochs:
Using 20 epochs means that the entire dataset will be iterated through the neural network 20 times during training. More epochs can potentially allow the model to learn more complex patterns from the data. However, increasing the number of epochs may also lead to overfitting, where the model becomes too specialized to the training data and performs poorly on new, unseen data. The choice of 20 epochs could be based on empirical observations that the model's validation performance tends to stabilize or converge within this range.

- Batch Size of 64:
The batch size determines the number of training samples that are propagated through the network before updating the model's weights. Smaller batch sizes (e.g., 32, 64) can lead to more frequent updates and potentially faster convergence, as the model updates its parameters more often. However, smaller batch sizes can also result in noisy gradients and slower training on hardware with high parallelism (like GPUs). Larger batch sizes (e.g., 128, 256) can provide more stable gradient estimates but might take longer to update the model.

In practice, the optimal values for epochs and batch size can vary depending on factors like the complexity of the dataset, the architecture of the model, available computing resources, and the presence of regularization techniques. It's common to try different values and monitor the model's performance on validation data to determine the best combination.

In [16]:
# Model trained with Tensorflow
model.fit(X_train_embed, Y_train_embed, validation_data = (X_test_embed,Y_test_embed), epochs=TRAINING_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/20
200/200 [==============================] - 7s 28ms/step - loss: 0.3298 - accuracy: 0.8364 - val_loss: 0.1947 - val_accuracy: 0.9169
Epoch 2/20
200/200 [==============================] - 4s 22ms/step - loss: 0.1513 - accuracy: 0.9408 - val_loss: 0.1992 - val_accuracy: 0.9214
Epoch 3/20
200/200 [==============================] - 5s 25ms/step - loss: 0.1095 - accuracy: 0.9586 - val_loss: 0.2228 - val_accuracy: 0.9180
Epoch 4/20
200/200 [==============================] - 5s 23ms/step - loss: 0.0886 - accuracy: 0.9691 - val_loss: 0.2376 - val_accuracy: 0.9074
Epoch 5/20
200/200 [==============================] - 4s 22ms/step - loss: 0.0702 - accuracy: 0.9763 - val_loss: 0.2890 - val_accuracy: 0.9152
Epoch 6/20
200/200 [==============================] - 5s 26ms/step - loss: 0.0563 - accuracy: 0.9807 - val_loss: 0.2806 - val_accuracy: 0.9151
Epoch 7/20
200/200 [==============================] - 4s 22ms/step - loss: 0.0409 - accuracy: 0.9878 - val_loss: 0.3277 - val_accuracy: 0.9131

In [17]:
# Save the model to a file
model.save('lstm_model_fake_news.h5')

### Testing

In [18]:
X_test_embed

array([[1943,  658, 4025, ...,    0,    0,    0],
       [1408,  708, 1291, ...,    0,    0,    0],
       [1408, 1456, 3047, ...,    0,    0,    0],
       ...,
       [ 695, 3024,  568, ...,    0,    0,    0],
       [2542, 4329, 2946, ...,    0,    0,    0],
       [1228,  242, 2062, ...,    0,    0,    0]], dtype=int32)

In [19]:
predictions = (model.predict(X_test_embed) > 0.5).astype("int32")

172/172 [==============================] - 2s 6ms/step


In [20]:
predictions

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

### Visualizations and metrics

In [21]:
accuracy_score(Y_test_embed,predictions)

0.9090411957710536

In [22]:
import tensorflow as tf
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Evaluate the model on test data
results = model.evaluate(X_test_embed, Y_test_embed)

loss = results[0]
accuracy = results[1]
precision = results[2]
recall = results[3]

print("Loss:", loss)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

172/172 [==============================] - 2s 4ms/step - loss: 0.5417 - accuracy: 0.9090 - precision: 0.8936 - recall: 0.8992
Loss: 0.5416796803474426
Accuracy: 0.909041166305542
Precision: 0.8935818076133728
Recall: 0.8991666436195374


### Improving the model

In [23]:
# Definir la función del modelo que deseas ajustar
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(VOCABULARY_SIZE, EMBEDDING_VECTOR_FEATURES, input_length=SENTENCE_LENGTH))
    model.add(keras.layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, default=0.3)))
    model.add(keras.layers.LSTM(hp.Int('units', min_value=50, max_value=200, step=50, default=100)))
    model.add(keras.layers.Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, default=0.3)))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Inicializar el optimizador de búsqueda
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Número de combinaciones a probar
    directory='my_dir',  # Directorio para almacenar resultados
    project_name='my_project'
)

# Realizar la búsqueda de hiperparámetros
tuner.search(X_train_embed, Y_train_embed, epochs=20, validation_split=0.2)

# Imprimir los mejores hiperparámetros encontrados
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Mejores hiperparámetros: {best_hps.values}")

Trial 10 Complete [00h 03m 44s]
val_accuracy: 0.9214843511581421

Best val_accuracy So Far: 0.9253906011581421
Total elapsed time: 00h 23m 46s
Mejores hiperparámetros: {'dropout': 0.38740223810710805, 'units': 100}


In [30]:
BEST_VOCABULARY_SIZE = 5000
BEST_EMBEDDING_VECTOR_FEATURES = 40
BEST_SENTENCE_LENGTH = 20
BEST_DROPOUT_RATE = 0.3874
BEST_LSTM_UNITS = 100
BEST_LEARNING_RATE = 0.001

In [31]:
model = Sequential()
model.add(Embedding(BEST_VOCABULARY_SIZE, BEST_EMBEDDING_VECTOR_FEATURES, input_length=BEST_SENTENCE_LENGTH))
model.add(Dropout(BEST_DROPOUT_RATE))
model.add(Bidirectional(LSTM(BEST_LSTM_UNITS, return_sequences=True)))
model.add(Bidirectional(LSTM(BEST_LSTM_UNITS)))
model.add(Dropout(BEST_DROPOUT_RATE))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))

In [32]:
# Compile the model
optimizer = Adam(learning_rate=BEST_LEARNING_RATE)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_4 (Dropout)         (None, 20, 40)            0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 20, 200)          112800    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                

In [33]:
# Train the model
history = model.fit(X_train_embed, Y_train_embed, validation_data = (X_test_embed,Y_test_embed), epochs=TRAINING_EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/20
200/200 [==============================] - 27s 104ms/step - loss: 0.3172 - accuracy: 0.8554 - val_loss: 0.1966 - val_accuracy: 0.9187
Epoch 2/20
200/200 [==============================] - 20s 102ms/step - loss: 0.1528 - accuracy: 0.9441 - val_loss: 0.1960 - val_accuracy: 0.9178
Epoch 3/20
200/200 [==============================] - 31s 156ms/step - loss: 0.1172 - accuracy: 0.9576 - val_loss: 0.2172 - val_accuracy: 0.9196
Epoch 4/20
200/200 [==============================] - 23s 113ms/step - loss: 0.0905 - accuracy: 0.9698 - val_loss: 0.2462 - val_accuracy: 0.9151
Epoch 5/20
200/200 [==============================] - 22s 108ms/step - loss: 0.0813 - accuracy: 0.9737 - val_loss: 0.2622 - val_accuracy: 0.9149
Epoch 6/20
200/200 [==============================] - 27s 137ms/step - loss: 0.0637 - accuracy: 0.9819 - val_loss: 0.2708 - val_accuracy: 0.9162
Epoch 7/20
200/200 [==============================] - 20s 99ms/step - loss: 0.0616 - accuracy: 0.9827 - val_loss: 0.2591 - val_acc

In [35]:
# Evaluate the model
y_pred = (model.predict(X_test_embed) > 0.5).astype("int32")
print("Confusion Matrix:")
print(confusion_matrix(Y_test_embed, y_pred))
print("Classification Report:")
print(classification_report(Y_test_embed, y_pred))

172/172 [==============================] - 3s 13ms/step
Confusion Matrix:
[[2814  272]
 [ 213 2187]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3086
           1       0.89      0.91      0.90      2400

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486



In [36]:
# Contar cuántos elementos son cero
count_zeros = y_pred.size - np.count_nonzero(y_pred)

print(f"Número de notificias verdaderas: {count_zeros}")
print(f"Número de notificias falsas: {np.count_nonzero(y_pred)}")

Número de notificias verdaderas: 3027
Número de notificias falsas: 2459


In [38]:
model.save('best_hiperparameters_lstm_model_fake_news.h5')

### Conclusions